#### This code attempts to reuse the VGG 16 convolutional layers to read the driver images. A small dense layer is then trained to classifies these VGG output features . Note that the dense layer is smaller than what was used by Jeremy, thus the accuracy stays around 74-76%.

In [1]:
from __future__ import print_function, division
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
%matplotlib inline
path="/home/ubuntu/nbs/data/state/"
#sample_path="/home/ubuntu/nbs/data/state/sample/"

In [3]:
from keras.preprocessing import image

In [4]:
batch_size=64

#### get a Directory Iterator  for the images in the training folder

In [5]:
datagen=image.ImageDataGenerator()

# flow from directory, with class names being categorical, image target size being  224 x 224, shuffle off
batches=datagen.flow_from_directory(path+'train',target_size=(224,224),class_mode='categorical',
shuffle=False,batch_size=batch_size)

Found 18009 images belonging to 10 classes.


#### Get a directory iterator for images in the validation folder

In [6]:
#get the validation set
val_batches=datagen.flow_from_directory(path+'valid',target_size=(224,224),class_mode='categorical',
shuffle=False,batch_size=batch_size)

Found 4415 images belonging to 10 classes.


In [7]:
# train labels
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18009 images belonging to 10 classes.
Found 4415 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [8]:
from vgg16 import Vgg16
#get the vgg model
vgg=Vgg16()
model=vgg.model

In [9]:
# find the  conv layers of vgg
last_conv_layer_id=[index for index,layer in enumerate(model.layers) if type(layer) is  Convolution2D][-1]
vgg_conv_layers=model.layers[:last_conv_layer_id+1]

In [10]:
# create a model from the vgg conv layers
conv_model=Sequential(vgg_conv_layers)

In [11]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]            
___________________________________________________________________________________________

In [12]:
# get numpy array of conv feature predictions for training data
conv_vgg_features=conv_model.predict_generator(batches,batches.nb_sample)

In [13]:
# get numpy array of conv feature predictions for val  data  
conv_val_vgg_features=conv_model.predict_generator(val_batches,val_batches.nb_sample)

In [15]:
#save to array
save_array(path+'results/conv_val_vgg_features.dat', conv_val_vgg_features)
save_array(path+'results/conv_vgg_feat.dat', conv_vgg_features)

In [16]:
# load array
conv_vgg_features = load_array(path+'results/conv_vgg_feat.dat')
conv_val_vgg_features = load_array(path+'results/conv_val_vgg_features.dat')
conv_val_vgg_features.shape

(4415, 512, 14, 14)

#### define function to return dense layers with batch norm and dropout

In [19]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=vgg_conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [20]:
bn_model=Sequential(get_bn_layers(.5))
bn_model.compile(Adam(lr=.001),loss='categorical_crossentropy',metrics=['accuracy'])
bn_model.fit(conv_vgg_features, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_vgg_features, val_labels))

Train on 18009 samples, validate on 4415 samples
Epoch 1/1
18009/18009 [==============================] - 7s - loss: 0.2633 - acc: 0.9300 - val_loss: 0.8522 - val_acc: 0.7273


In [21]:
bn_model.fit(conv_vgg_features, trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_vgg_features, val_labels))

Train on 18009 samples, validate on 4415 samples
Epoch 1/4
18009/18009 [==============================] - 7s - loss: 0.0211 - acc: 0.9976 - val_loss: 0.8259 - val_acc: 0.7586
Epoch 2/4
18009/18009 [==============================] - 7s - loss: 0.0109 - acc: 0.9984 - val_loss: 0.9818 - val_acc: 0.7280
Epoch 3/4
18009/18009 [==============================] - 7s - loss: 0.0065 - acc: 0.9993 - val_loss: 0.8985 - val_acc: 0.7465
Epoch 4/4
18009/18009 [==============================] - 7s - loss: 0.0047 - acc: 0.9994 - val_loss: 0.8517 - val_acc: 0.7640


In [24]:
bn_model.optimizer.lr=0.00001
bn_model.fit(conv_vgg_features, trn_labels, batch_size=batch_size, nb_epoch=8, 
             validation_data=(conv_val_vgg_features, val_labels))

Train on 18009 samples, validate on 4415 samples
Epoch 1/8
18009/18009 [==============================] - 7s - loss: 0.0052 - acc: 0.9988 - val_loss: 0.9743 - val_acc: 0.7320
Epoch 2/8
18009/18009 [==============================] - 7s - loss: 0.0044 - acc: 0.9988 - val_loss: 1.0065 - val_acc: 0.7373
Epoch 3/8
18009/18009 [==============================] - 7s - loss: 0.0039 - acc: 0.9993 - val_loss: 1.7080 - val_acc: 0.6360
Epoch 4/8
18009/18009 [==============================] - 7s - loss: 0.0048 - acc: 0.9986 - val_loss: 1.0699 - val_acc: 0.7749
Epoch 5/8
18009/18009 [==============================] - 7s - loss: 0.0047 - acc: 0.9986 - val_loss: 1.1949 - val_acc: 0.7343
Epoch 6/8
18009/18009 [==============================] - 7s - loss: 0.0057 - acc: 0.9983 - val_loss: 1.2392 - val_acc: 0.7352
Epoch 7/8
18009/18009 [==============================] - 7s - loss: 0.0056 - acc: 0.9982 - val_loss: 1.1419 - val_acc: 0.7642
Epoch 8/8
18009/18009 [==============================] - 7s - loss: 0

#### Since we are overfitting the training set rapidly, increasing the dropout

In [26]:
bn_model=Sequential(get_bn_layers(.6))
bn_model.compile(Adam(lr=.00001),loss='categorical_crossentropy',metrics=['accuracy'])
bn_model.fit(conv_vgg_features, trn_labels, batch_size=batch_size, nb_epoch=6, 
             validation_data=(conv_val_vgg_features, val_labels))

Train on 18009 samples, validate on 4415 samples
Epoch 1/6
18009/18009 [==============================] - 7s - loss: 2.6826 - acc: 0.2587 - val_loss: 1.7271 - val_acc: 0.3900
Epoch 2/6
18009/18009 [==============================] - 7s - loss: 1.3250 - acc: 0.5712 - val_loss: 1.2886 - val_acc: 0.5604
Epoch 3/6
18009/18009 [==============================] - 7s - loss: 0.7935 - acc: 0.7386 - val_loss: 1.0837 - val_acc: 0.6507
Epoch 4/6
18009/18009 [==============================] - 7s - loss: 0.5437 - acc: 0.8268 - val_loss: 1.0052 - val_acc: 0.6838
Epoch 5/6
18009/18009 [==============================] - 7s - loss: 0.3895 - acc: 0.8803 - val_loss: 0.9201 - val_acc: 0.7096
Epoch 6/6
18009/18009 [==============================] - 7s - loss: 0.3075 - acc: 0.9100 - val_loss: 0.8848 - val_acc: 0.7264


#### Increasing learning rate to go faster

In [27]:
bn_model.optimizer.lr=0.001
bn_model.fit(conv_vgg_features, trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_vgg_features, val_labels))

Train on 18009 samples, validate on 4415 samples
Epoch 1/4
18009/18009 [==============================] - 7s - loss: 0.2388 - acc: 0.9319 - val_loss: 0.8454 - val_acc: 0.7368
Epoch 2/4
18009/18009 [==============================] - 7s - loss: 0.2026 - acc: 0.9424 - val_loss: 0.8228 - val_acc: 0.7413
Epoch 3/4
18009/18009 [==============================] - 7s - loss: 0.1750 - acc: 0.9538 - val_loss: 0.8251 - val_acc: 0.7400
Epoch 4/4
18009/18009 [==============================] - 7s - loss: 0.1463 - acc: 0.9617 - val_loss: 0.8387 - val_acc: 0.7420


#### A classification accuracy of 74.2% was achieved